In [1]:
!pip install transformers datasets torchaudio tensorflow librosa matplotlib 
!pip install datasets
!pip install datasets
!pip install tf-keras
!pip install transformers[torch]
#!pip install numpy<2.0
#!pip install torch==2.1.0+cu121 torchvision==0.16.0+cu121 torchaudio --index-url https://download.pytorch.org/whl/cu121

  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.10.1
    Uninstalling huggingface-hub-0.10.1:
      Successfully uninstalled huggingface-hub-0.10.1


In [32]:
import os
import pandas as pd
import torch
import torchaudio
import torchaudio.transforms as T
from sklearn.model_selection import train_test_split
from datasets import Dataset, Audio
import transformers

import matplotlib.pyplot as plt


In [36]:
# Paths
audio_dir = r"C:\Users\cheny\Downloads\common_voice\cv-valid-train"
csv_file_path = r"C:\Users\cheny\Downloads\common_voice\cv-valid-train.csv"

# Load CSV
df = pd.read_csv(csv_file_path)

# Construct full paths to MP3 files
df["path"] = df["filename"].apply(lambda x: os.path.join(audio_dir, x))

# Check column names
print(df.columns)

# Train-validation split
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))

# Cast audio column to `Audio`
train_dataset = train_dataset.cast_column("path", Audio())
val_dataset = val_dataset.cast_column("path", Audio())


Index(['filename', 'text', 'up_votes', 'down_votes', 'age', 'gender', 'accent',
       'duration', 'path'],
      dtype='object')


In [37]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import Dataset
import torch
import torchaudio
import logging

logging.basicConfig(filename="preprocessing_errors.log", level=logging.WARNING)

def preprocess(example):
    try:
        audio = example["path"]
        waveform = audio["array"]
        sampling_rate = audio["sampling_rate"]

        # Resample if needed
        if sampling_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
            waveform = resampler(torch.tensor(waveform).float()).numpy()

        # Encode input
        input_values = processor(waveform, sampling_rate=16000, return_tensors="pt").input_values[0]

        # Encode label
        labels = processor.tokenizer(
            example["text"],
            add_special_tokens=False
        ).input_ids

        return {
            "input_values": input_values,
            "labels": torch.tensor(labels)
        }

    except Exception as e:
        logging.warning(f"Failed to process: {example.get('path', 'unknown')} — {str(e)}")
        return None  # return nothing if it fails

# Manually process train_dataset row-by-row
processed_train = []
for i, example in enumerate(train_dataset):
    result = preprocess(example)
    if result is not None:
        processed_train.append(result)

processed_val = []
for i, example in enumerate(val_dataset):
    result = preprocess(example)
    if result is not None:
        processed_val.append(result)




In [ ]:
# Create new datasets
train_dataset = Dataset.from_list(processed_train)
val_dataset = Dataset.from_list(processed_val)

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [6]:
import torch
# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Check if CUDA is available
if torch.cuda.is_available():
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU: NVIDIA GeForce RTX 3070 Ti Laptop GPU


In [4]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    evaluation_strategy="epoch",
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_dir="./logs",
    save_strategy="epoch",
    report_to="none"
)


torch.Size([1, 128, 156])


C:\Users\cheny\anaconda3\envs\py311\Lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [5]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor,
    data_collator=data_collator
)

trainer.train()

C:\Users\cheny\anaconda3\envs\py311\Lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


{'input_values': tensor([[[0.0015, 0.0015, 0.0015,  ..., 0.0015, 0.0015, 0.0015]]])}


In [15]:
model.save_pretrained("./wav2vec2-large-960h-cv")
processor.save_pretrained("./wav2vec2-large-960h-cv")

Sample paths:
 0    C:\Users\cheny\Downloads\common_voice\cv-valid...
1    C:\Users\cheny\Downloads\common_voice\cv-valid...
2    C:\Users\cheny\Downloads\common_voice\cv-valid...
3    C:\Users\cheny\Downloads\common_voice\cv-valid...
4    C:\Users\cheny\Downloads\common_voice\cv-valid...
Name: path, dtype: object


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
import matplotlib.pyplot as plt

logs = trainer.state.log_history
train_loss = [log["loss"] for log in logs if "loss" in log]
eval_loss = [log["eval_loss"] for log in logs if "eval_loss" in log]

plt.plot(train_loss, label="Train Loss")
plt.plot(eval_loss, label="Eval Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Training vs Evaluation Loss")
plt.grid(True)
plt.show()


Map:   0%|          | 0/137043 [00:00<?, ? examples/s]

TypeError: Wav2Vec2CTCTokenizer(name_or_path='facebook/wav2vec2-large-960h', vocab_size=32, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	1: AddedToken("<s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	2: AddedToken("</s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	3: AddedToken("<unk>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
}
) got multiple values for keyword argument 'return_attention_mask'

In [ ]:
import os
import pandas as pd
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from jiwer import wer

# Load the model and processor
processor = Wav2Vec2Processor.from_pretrained("wav2vec2-large-960h-cv")
model = Wav2Vec2ForCTC.from_pretrained("wav2vec2-large-960h-cv")

# Load test data
test_csv_file_path = r'C:\Users\cheny\Downloads\common_voice\cv-valid-test.csv'
test_df = pd.read_csv(test_csv_file_path)



# Directory containing test audio files
test_audio_dir = r'C:\Users\cheny\Downloads\common_voice\cv-valid-test\'

# Initialize list to store results
results = []

# Loop over each row in the CSV to get the test files and their ground truth
for index, row in test_df.iterrows():
    audio_file = os.path.join(test_audio_dir, row['filename'])
    ground_truth = row['transcription']

    # Transcribe the audio file
    transcription = transcribe_audio(audio_file)
    
    # Calculate WER
    wer_score = wer(ground_truth, transcription)

    # Append the results
    results.append({
        'filename': row['filename'],
        'ground_truth': ground_truth,
        'transcription': transcription,
        'wer': wer_score
    })

# Convert results into a DataFrame for easy inspection
results_df = pd.DataFrame(results)

# Display the results (or save to a file)
print(results_df)

# Optionally, save results to a CSV file
results_df.to_csv('transcription_results.csv', index=False)

# Calculate the min, max, and average WER
min_wer = results_df['wer'].min()
max_wer = results_df['wer'].max()
average_wer = results_df['wer'].mean()

# Print the min, max, and average WER
print(f"Min WER: {min_wer:.4f}")
print(f"Max WER: {max_wer:.4f}")
print(f"Average WER: {average_wer:.4f}")


Adapted from ChatGPT: 

Explanation for Preprocessing, Tokenizer, Feature Extraction, and Pipeline Processes

In this section, I'll explain the choices I made for preprocessing, tokenizer, feature extraction, and pipeline processes for fine-tuning an Automatic Speech Recognition (ASR) model using the Common Voice dataset. Additionally, I'll describe the hyperparameters used for training and what the training/validation metrics mean.

1. Preprocessing
Why Preprocessing Is Necessary: Preprocessing ensures that the audio data is in a suitable format for the model to understand. It involves steps like loading the audio, resampling to a fixed sample rate, and ensuring consistency in the audio input format.

The original dataset contains audio with varying sample rates. To ensure consistency, I resampled all audio files to a fixed sample rate of 16kHz. This is the standard used by many ASR models, including Wav2Vec2, as it balances computational efficiency and audio quality.

2. Tokenizer
Why We Use a Tokenizer: The tokenizer converts text data (the transcription of the audio) into numerical representations that the model can work with. Since Wav2Vec2 uses CTC (Connectionist Temporal Classification) loss, it requires a tokenizer that can map the transcription to labels and vice versa.

Wav2Vec2Processor: This processor from Hugging Face's transformers library automatically handles both feature extraction from audio and tokenization of transcriptions.

The processor:
Tokenizes the transcription (text) into input_ids.
Converts audio into features that are suitable for the Wav2Vec2 model.

3. Feature Extraction
Why Feature Extraction Is Necessary:
Wav2Vec2 works directly with raw audio waveforms, but we can also extract spectrograms to enhance performance in some cases.
However, in this case, we will use the Wav2Vec2Processor, which includes feature extraction as part of the preprocessing pipeline. The processor:
Extracts Mel-frequency cepstral coefficients (MFCCs) or raw waveforms.
Normalizes the input for consistency in processing.
Converts raw waveforms into suitable input for Wav2Vec2.

4. Pipeline
The pipeline consists of several steps that convert raw data into a form suitable for training a deep learning model.

Pipeline Components:
Load Audio: Use torchaudio to read audio files.
Resample Audio: Resample all audio to 16kHz to maintain consistency.
Feature Extraction: Convert the waveform to Mel Spectrogram or raw audio features.
Tokenization: Use the Wav2Vec2Processor to handle both feature extraction and transcription tokenization.
Model Training: Feed the processed audio and transcription into the model for training.
Validation: Split the dataset into training (70%) and validation (30%) sets. Validate on a separate subset during training.




5. Hyperparameters for Model Training
Below are the selected hyperparameters and the reasoning behind their configuration for fine-tuning wav2vec2-large-960h on the Common Voice dataset:

Learning Rate (2e-5):
A low learning rate is recommended when fine-tuning pre-trained models like Wav2Vec2 to avoid overwriting pre-trained knowledge. A value of 2e-5 strikes a balance between learning efficiently and preserving previously acquired representations, as suggested by the official Hugging Face documentation.

Batch Size (8):
We use a batch size of 8 per device, which provides a good trade-off between memory efficiency and training stability. Since audio data can vary in length and can be memory-intensive, especially in 16 kHz resolution, this size ensures that training remains within GPU memory limits.

Number of Epochs (3):
A small number of training epochs is generally sufficient for fine-tuning on a moderately sized dataset. We selected 3 epochs to allow enough updates for learning, while keeping training time reasonable. Further tuning or early stopping could be applied if needed.

Evaluation Strategy ("epoch"):
We evaluate the model at the end of every epoch to track performance on the validation set. This provides insights into whether the model is improving and helps detect overfitting early.

Save Strategy ("epoch"):
Instead of saving every few steps, the model is saved at the end of each epoch. This is less frequent but aligns well with our evaluation frequency and simplifies checkpoint management.

Logging Steps (10):
Training logs are recorded every 10 steps. This provides enough granularity to visualize learning trends in real-time without overwhelming the log output.

Data Collator (DataCollatorCTCWithPadding):
We use Hugging Face’s DataCollatorCTCWithPadding to dynamically pad audio and label sequences during training. This ensures efficient and correct batching of variable-length input audio and transcriptions.

6. Visualizing Training and Validation Metrics
After training, we visualize the loss metrics to understand how the model learned over time and evaluate the quality of training and generalization.

Loss Curves:
Both training and validation losses are plotted across epochs. This helps us visually track how well the model is fitting the data.

Interpretation of Loss Curves
Training Loss:
A steadily decreasing training loss typically indicates that the model is learning effectively. In our case, training loss decreased across epochs, showing that the model was optimizing its internal parameters.

Validation Loss:
The validation loss reflects how well the model generalizes to unseen data. Ideally, it should decrease in parallel with training loss. If it increases while training loss decreases, it’s a signal that the model is beginning to overfit.

Convergence:
In our training, both curves show a consistent downward trend, suggesting that the model is converging.

Overfitting Signs:
We monitor for signs of overfitting, such as a divergence between training and validation loss. If such divergence had occurred, we could reduce the number of epochs or implement early stopping strategies to mitigate it.